In [1]:
import pandas as pd
from sodapy import Socrata

In [2]:
Socrata.get?

Signature: Socrata.get(self, dataset_identifier, content_type='json', **kwargs)
Docstring:
Read data from the requested resource. Options for content_type are json,
csv, and xml. Optionally, specify a keyword arg to filter results:

    select : the set of columns to be returned, defaults to *
    where : filters the rows to be returned, defaults to limit
    order : specifies the order of results
    group : column to group results on
    limit : max number of results to return, defaults to 1000
    offset : offset, used for paging. Defaults to 0
    q : performs a full text search for a value
    query : full SoQL query string, all as one parameter
    exclude_system_fields : defaults to true. If set to false, the
        response will include system fields (:id, :created_at, and
        :updated_at)

More information about the SoQL parameters can be found at the official
docs:
    http://dev.socrata.com/docs/queries.html

More information about system fields can be found here:
    h

In [3]:
# Establish queries
# https://dev.socrata.com/docs/queries/

# cols = [
#     # 'year',
#     'date',
#     'statefips',
#     'countyfips',
#     'ctfips',
#     # 'latitude',
#     # 'longitude',
#     'ds_pm_pred',
#     'ds_pm_stdd',
# ]

# filt = [
#     # "year = '2014'",
#     "RIGHT(date, 1) = '4'", # same as year == 2014, but works w/o selecting year
#     "statefips IN ('6', '48', '17')", # in CA, TX, or IL
# ]

# Establish queries
# https://dev.socrata.com/docs/queries/

cols = "date, statefips, countyfips, ctfips, ds_pm_pred, ds_pm_stdd"

filt = "year = '2014' AND statefips IN ('6', '48', '17')" #RIGHT(date, 1) = '4'

In [5]:
# https://data.cdc.gov/Environmental-Health-Toxicology/Daily-Census-Tract-Level-PM2-5-Concentrations-2011/fcqm-xrf4

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cdc.gov", None)

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("fcqm-xrf4", select=cols, where=filt, limit=10000000)

client.close()

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

print(results_df.shape)
results_df.head(5)

(5991475, 6)


,date,statefips,countyfips,ctfips,ds_pm_pred,ds_pm_stdd
0,01JAN2014,48,48061,48061014100,13.9979,14.8386
1,01JAN2014,48,48061,48061013309,14.2144,14.9526
2,01JAN2014,48,48061,48061013308,14.874,15.7786
3,01JAN2014,48,48061,48061013307,14.7068,15.4085
4,01JAN2014,48,48061,48061013306,14.6796,16.2741


In [6]:
df_dtypes = {
    'year': 'int16', # can be dropped
    'date': 'category', 
    'statefips': 'int8', 
    'countyfips': 'int32', 
    'ctfips': 'int64', 
    'latitude': 'float32',
    'longitude': 'float32', 
    'ds_pm_pred': 'float32', 
    'ds_pm_stdd': 'float32',
}

for k, v in df_dtypes.items():
    if k in results_df.columns:
        if k == 'date':
            # continue
            results_df[k] = pd.to_datetime(results_df[k], format='%d%b%Y')
        else:
            results_df[k] = results_df[k].astype(v)

results_df.dtypes

date          datetime64[ns]
statefips               int8
countyfips             int32
ctfips                 int64
ds_pm_pred           float32
ds_pm_stdd           float32
dtype: object

In [9]:
# Print size of each col in RAM (units are MB)
results_df.memory_usage(index=True, deep=True) / (1024**2)

Index          0.000122
date          45.711327
statefips      5.713916
countyfips    22.855663
ctfips        45.711327
ds_pm_pred    22.855663
ds_pm_stdd    22.855663
dtype: float64

In [11]:
import dask.dataframe as dd

ddf = dd.from_pandas(results_df, npartitions=60) # of ~100,000 rows each


In [13]:
ddf.to_hdf('cdc_data.h5', key='pm')

['cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5',
 'cdc_data.h5']

In [17]:
ddf.to_hdf('cdc_data_comp.h5', key='pm', complib='lzo', complevel=1, format='table')

['cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_comp.h5',
 'cdc_data_co

In [14]:
ddf.to_parquet('cdc_data.parquet.gzip', compression='gzip')

(None,)

In [16]:
ddf.dtypes

date          datetime64[ns]
statefips               int8
countyfips             int32
ctfips                 int64
ds_pm_pred           float32
ds_pm_stdd           float32
dtype: object